### RAG (Retrieval Augmented Generation ) 
This notebook code reads a .pdf file, chunk and store into a db.
On requesting for an answer to the question, this code read from the stored db + model knowledge to answer.

-------------------------------------------------------------------------------------- -
.pdf --> chunk --> store in chorma db --> retrieve data from db (similarity search) --> create prompt template --> LLM run
-------------------------------------------------------------------------------------- -
Context is similarity search result from custom knowledge  base

LLM Model = ggml-gpt4all-j-v1.3-groovy

In [15]:
import textwrap
import chromadb

from langchain.llms import GPT4All
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain import PromptTemplate, LLMChain


from langchain.vectorstores import Chroma
from chromadb.config import Settings

Download the model from https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin (Date : 10/17/2023)
File size of the model is 3gb.

In [26]:
CHROMA_SETTINGS = Settings(
    persist_directory="../db/mychroma",
    anonymized_telemetry=False,
    allow_reset=True
)
EMBEDDINGS_MODEL_NAME="all-MiniLM-L6-v2"
model_path = '../models/ggml-gpt4all-j-v1.3-groovy.bin'
vector_db_path='../db/'

text_wrapper = textwrap.TextWrapper(width=100)

Initialize GPT4All, Embeddings, Chroma db client and db and text Splitter

In [17]:
llm = GPT4All(model=model_path, max_tokens=2000, verbose=True, 
              allow_download=False, repeat_last_n=0)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)

Found model file at  ../models/ggml-gpt4all-j-v1.3-groovy.bin


Load and split files, source is NXT advanced programming 

In [27]:
#pdf_loader = PyPDFLoader("../data/advanced_programmingforprint.pdf")
pdf_loader = PyPDFLoader("C:/Users/ppoob/Downloads/mc_2_patterns.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
docs = pdf_loader.load_and_split(text_splitter=text_splitter)
len(docs)   

15

In [28]:
# Store into db
chroma_db = Chroma.from_documents( documents=docs, embedding=embeddings, persist_directory="../db/", 
             client_settings= CHROMA_SETTINGS) #TODO - upsert
chroma_db.persist()


### Similarity Search.

In [29]:
question = "What is data Wires  ? "
question = "what are patterns ?"
matched_docs = chroma_db.similarity_search(question)
matched_docs

[Document(page_content='Page | 1  2023-2024 Math Challenge   \n \n \nFirst Name:  __________________ ___ Last Name:  ________________________________  Grade:  _____ ____ \nTeacher:  _____________________ __ Parent’s email:  ___________________________________________ __ \n \nPatterns  \n \nIn Math, a pattern means things like numbers, shapes, or colors that repeat in a certain way. When numbers follow a \nspecial rule and go together, that\'s called a pattern too. Sometimes, we also call patterns "sequences."', metadata={'page': 0, 'source': 'C:/Users/ppoob/Downloads/mc_2_patterns.pdf'}),
 Document(page_content='Page | 1  2023-2024 Math Challenge   \n \n \nFirst Name:  __________________ ___ Last Name:  ________________________________  Grade:  _____ ____ \nTeacher:  _____________________ __ Parent’s email:  ___________________________________________ __ \n \nPatterns  \n \nIn Math, a pattern means things like numbers, shapes, or colors that repeat in a certain way. When numbers follow

### Retrieve from Database
- Open DB
- Similarity search from persisted DB
- Create context

In [30]:
#retrieve from vector db
persisted_db = Chroma(persist_directory=CHROMA_SETTINGS.persist_directory, 
                            embedding_function=embeddings, 
                            client_settings=CHROMA_SETTINGS)
# Only retrieve documents that have a relevance score
# Above a 0.8 threshold
# source_vector_store_retriever = persisted_db.as_retriever(search_type="similarity_score_threshold",
#                                             search_kwargs={'score_threshold': 0.8})

source_vector_store_retriever = persisted_db.as_retriever(search_type="mmr")

matched_docs = source_vector_store_retriever.get_relevant_documents(question)

print(len(matched_docs))
context = ''
for doc in matched_docs:
    context = context + doc.page_content

print(text_wrapper.fill(context))

4
Page | 1  2023-2024 Math Challenge        First Name:  __________________ ___ Last Name:
________________________________  Grade:  _____ ____  Teacher:  _____________________ __ Parent’s
email:  ___________________________________________ __    Patterns     In Math, a pattern means
things like numbers, shapes, or colors that repeat in a certain way. When numbers follow a  special
rule and go together, that's called a pattern too. Sometimes, we also call patterns "sequences."a.
b.            Consecutive numbers  are the numbers that continuously follow each other, one after
another in a regular counting order or in the order from least to greatest .            a.     b.
2  6  10     3  9  15 18     4  10  16  22      Input  Output   1 5  2 7  3 9  4 11  … …  10 ?2.
Find each number that is hiding under each  flower to complete the pattern.   10 20  40   60 70 80
100         3. Lilian crossed out all even numbers from the list. Maria then crossed out every fifth
number from the origina

Create Prompt (Zero-shot) template, input variables
- context
- question

Output - llm chain not returning full response, where as RetrievalQA.from_chain_type is retuning full expected response.

In [31]:
#persisted_db.delete_collection()
#persisted_db.get()

In [33]:
print(f"Context : {context}")
template = """
Please use the following below context to answer questions. If you don't know the answer, just say that you don't know. don't try to make up an answer.
Context: {context}
Question: {question}
Answer: """

prompt = PromptTemplate(input_variables=['context','question'], template=template).partial(context=context)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=False)
response = llm_chain.run(question)
print(text_wrapper.fill(response))


Context : Page | 1  2023-2024 Math Challenge   
 
 
First Name:  __________________ ___ Last Name:  ________________________________  Grade:  _____ ____ 
Teacher:  _____________________ __ Parent’s email:  ___________________________________________ __ 
 
Patterns  
 
In Math, a pattern means things like numbers, shapes, or colors that repeat in a certain way. When numbers follow a 
special rule and go together, that's called a pattern too. Sometimes, we also call patterns "sequences."a.                                                                         b. 
 
 
 
 
 
Consecutive numbers  are the numbers that continuously follow each other, one after 
another in a regular counting order or in the order from least to greatest .   
 
 
 
 
a.  
 
b.  
   2  6  10 
 
 3  9  15 18 
 
 4  10  16  22 
 
 
Input  Output  
1 5 
2 7 
3 9 
4 11 
… … 
10 ?2. Find each number that is hiding under each  flower to complete the pattern.  
10 20  40  
60 70 80  100 
  
   
3. Lilian crossed out al

### Leading words example

The model is forced to break down its solution into multiple, more manageable steps rather than being allowed to just hazard a guess
```
“think step by step”
```

In [20]:

template = """
Please use the following below context to answer questions. If you don't know the answer, just say that you don't know. don't try to make up an answer.
Context: {context}

Question: {question}
Answer: think step by step"""

prompt = PromptTemplate(input_variables=['context','question'], 
                        template=template).partial(context=context)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=False)
llm_chain.deb
response = llm_chain.run(question)
print(response)


, what is data wire? 

Data wire is a communication channel that allows data to be passed between different parts of a program. It is a way to pass information between different blocks of code in a program.

In the context of the program, data wires are used to pass information between the loop and the motors. The loop is used to control the motor power level, and the motors are controlled by the data wires.

The program also uses variables to store information. In this case, the program uses two variables, "LightValue" and "MotorPower", to store information. The program uses these variables to control the motor power level and to display the value of the variables in the NXT window.

Overall, the program uses data wires and variables to control the motor power level and display the value of the variables in the NXT window.


In [25]:

question = "Why did banana cross the road"
template = """Please use the following below context to answer questions. If you don't find answer form the context, just say that I don't know. DO NOT TRY TO MAKE UP.
Context: {context}

Question: {question}
Answer: think step by step"""

prompt = PromptTemplate(input_variables=['context','question'], 
                        template=template).partial(context=context)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=False)
response = llm_chain.run(question)
print(response)




1. Banana is on the road
2. Banana wants to cross the road
3. Banana sees a car coming
4. Banana decides to cross the road
5. Banana crosses the road
6. Banana is safe
7. Banana is happy
8. Banana is done
9. Banana is happy
10. Banana is happy
11. Banana is happy
12. Banana is happy
13. Banana is happy
14. Banana is happy
15. Banana is happy
16. Banana is happy
17. Banana is happy
18. Banana is happy
19. Banana is happy
20. Banana is happy
21. Banana is happy
22. Banana is happy
23. Banana is happy
24. Banana is happy
25. Banana is happy
26. Banana is happy
27. Banana is happy
28. Banana is happy
29. Banana is happy
30. Banana is happy
31. Banana is happy
32. Banana is happy
33. Banana is happy
34. Banana is happy
35. Banana is happy
36. Banana is happy
37. Banana is happy
38. Banana is happy
39. Banana is happy
40. Banana is happy
41. Banana


### Search without prompt

In [19]:

chunk = 4
# recreat the retriever. using similar search above failed to retrieve source document
# UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8

# Only get the single most similar document from the dataset
retriever = persisted_db.as_retriever(search_kwargs={"k": chunk})
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                           retriever=retriever, return_source_documents=True)
result = retrieval_qa(question) 

answer, source = result['result'], result['source_documents']

print(text_wrapper.fill(f'Answer : {answer}'))
print('\n')
print(text_wrapper.fill(f'Source : {source}' ))


Answer :  Data Wire is a communication channel that allows data to be passed between different parts
of a program. It is similar to variables, but it is not stored in memory. Data wires are connected
at data terminals, which are normally hidden. They can be used to pass information between blocks,
and they can be used to pass information between different programs.


Source : [Document(page_content='9/13/2011\n7Data Wires\n\uf097We use Data Wires to pass
\ninformation around inside of a program.  This is easier than using variables and accomplishes much
of the same function\n\uf097Data wires can go between blocks and are connected at the Data T
erminals (normally hidden)\n\uf097Shown here are the same move blocks with the Data T erminal hidden
and shown\nPress here to \nopenPress here to \nclose\nLinking the Loop to the Motors\n\uf097Lets run
a loop from 0 to 100\n\uf097And lets make the Motor Power level equal the Loop', metadata={'page':
6, 'source': '../data/advanced_programmingforp